# Libraries

In [1]:
from models import  unet, finefeaturepath
import tensorflow as tf 
print(tf.config.list_physical_devices('GPU') )
import keras 
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns 
import cv2
import os
import PIL
import glob


2024-05-10 09:39:25.519500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 09:39:27.142997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-10 09:39:30.234673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 09:39:30.605717: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 09:39:30.605876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Data preprocessing

In [2]:
class_names = ['HandWrittenText', 'PrintedText', 'BackGround', 'Overlap']

In [3]:
train_images = []

for directory_path in glob.glob("/mnt/C4B416C4B416B93E/Data_Science/Segmentation/Data/train/crop/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, (256, 256))
        train_images.append(img)

# Convert list to array for machine learning process
train_images = np.array(train_images)

In [4]:
train_masks = []
for directory_path in glob.glob("/mnt/C4B416C4B416B93E/Data_Science/Segmentation/Data/train/label/"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (256, 256))
        train_masks.append(mask)

train_masks = np.array(train_masks)

In [5]:
train_images.shape

(5169, 256, 256)

In [6]:
train_masks.shape

(5169, 256, 256)

In [7]:
np.unique(train_masks)

array([ 29,  76, 149, 225], dtype=uint8)

In [8]:
from sklearn.preprocessing import  LabelEncoder 
labelencoder = LabelEncoder()

n, h, w = train_masks.shape 
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([0, 1, 2, 3])

In [9]:
train_images = np.expand_dims(train_images, axis=3)
train_images = train_images/255.0

In [10]:
train_images.shape

(5169, 256, 256, 1)

In [11]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [12]:
train_masks_input.shape

(5169, 256, 256, 1)

In [13]:
from sklearn.model_selection import  train_test_split 
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10)

In [14]:
# further split training data to a smaller subset for quick testing model
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size=0.2)

In [15]:
print("Classes values in the dataset are ... ", np.unique(y_train))

Classes values in the dataset are ...  [0 1 2 3]


In [16]:
n_classes = 4

In [17]:
from keras.utils import  to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes ))

In [18]:
train_masks_cat.shape

(3721, 256, 256, 4)

In [19]:
y_train_cat.shape

(3721, 256, 256, 4)

In [20]:
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [21]:
y_test_cat.shape

(517, 256, 256, 4)

In [22]:
# from sklearn.utils import  class_weight
# class_weights = class_weight.compute_class_weight('balanced', np.unique(train_masks_reshaped_encoded), train_masks_reshaped_encoded)
# print("Class weights are....", class_weights)

# Compile Model

In [23]:
inputs = tf.keras.layers.Input(shape=(256,256,3))
u = unet(inputs)
f = finefeaturepath(inputs)
c = tf.keras.layers.concatenate([u,f])
outputs = tf.keras.layers.Conv2D(kernel_size=(1,1), filters=4, activation='softmax')(c)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

2024-05-10 09:40:14.916398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 09:40:14.916590: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 09:40:14.916712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit(X_train, y_train_cat,
                    batch_size=16,
                    verbose=1,
                    epochs=50,
                    validation_data = (X_test, y_test_cat),
                    #class_weight = class_weights
                    shuffle=False)

: 